In [1]:
from scipy.stats import multivariate_normal as mvn
from scipy.stats import norm
from scipy.stats import chi2
import numpy as np
import pandas as pd
from scipy import optimize
import math

In [2]:
def BondPrice(P, T, c_rate, y):
    c = c_rate * P
    
    B = P / math.pow(1 + y, T)
    for i in range(1, T + 1):
        B += (c / math.pow(1 + y, i))

    return B


def YTM(B, P, T, c_rate):
    
    def BondPriceHelper(P, T, c_rate, y):
        c = c_rate * P

        B = P / math.pow(1 + y, T)
        for i in range(1, T + 1):
            B += (c / math.pow(1 + y, i))

        return B

    def YTMSolver(y):
        lhs = BondPriceHelper(P, T, c_rate, y)
        rhs = B
        return (lhs - rhs) * 100000
    
    ytm = optimize.fsolve(YTMSolver, 0.1)
    
    return ytm[0]


def myBondCalc(B, P, T, c_rate, y):
    c = c_rate * P

    _cOfp = 0
    _cOfp2 = 0
    for i in range(1, T + 1):
        _cOfp += (c / math.pow(1 + y, i) * i)
        _cOfp2 += (c / math.pow(1 + y, i) * i * (i + 1))

    MacDur = (_cOfp + (P / math.pow(1 + y, T)) * T) / B
    ModDur = MacDur / (1 + y)

    _d = _cOfp2 + P / math.pow(1 + y, T) * T * (T + 1)
    ModCov = _d / (B * math.pow(1 + y, 2))
    
    return [MacDur, ModDur, ModCov]



def myPriceEuroOption(S, K, std, q, rf, T, ty):
    _d1u = math.log(S / K) + (rf - q + std ** 2 / 2) * T
    _d1d = std * math.sqrt(T)
    d1 = _d1u / _d1d
    d2 = d1 - _d1d

    if ty == "call":
        p = S * math.exp(-q * T) * norm.cdf(d1) - K * math.exp(-rf * T) * norm.cdf(d2)
        delta = math.exp(-q * T) * norm.cdf(d1)
    if ty == "put":
        p = K * math.exp(-rf * T) * norm.cdf(-d2) - S * math.exp(-q * T) * norm.cdf(-d1)
        delta = -math.exp(-q * T) * norm.cdf(-d1)
    gamma = norm.pdf(d1) / (S * std * math.sqrt(T))
    
    return [p, delta, gamma]


In [12]:
# ZCB VaR

P = 100
T = 5
y = 0.06
c_rate = 0
B = BondPrice(P, T, c_rate, y)

d_y = 20 / 10000

std_annu = 0.02 * math.sqrt(12)
z_alpha = norm.ppf(0.05)



[MacDur, ModDur, ModCov] = myBondCalc(B, P, T, c_rate, y)

# monthly duration-based VaR
d_t = 1 / 12
VaR_D = - (B * y * d_t + z_alpha * ModDur * B * std_annu * math.sqrt(d_t))
print("monthly Duration-based VaR:", round(VaR_D, 4))

# monthly duration-convexity-based VaR
VaR_DC = - (B * y * d_t +\
            z_alpha * (ModDur * B * std_annu * math.sqrt(d_t) + z_alpha/2 * ModCov * B * std_annu**2 * d_t))
print("monthly Duration-Convexity-based VaR:", round(VaR_DC, 4))
print()


# daily duration-based VaR
d_t = 1 / 252
VaR_D = - (B * y * d_t + z_alpha * ModDur * B * std_annu * math.sqrt(d_t))
print("daily Duration-based VaR:", round(VaR_D, 4))

# daily duration-convexity-based VaR
VaR_DC = - (B * y * d_t +\
            z_alpha * (ModDur * B * std_annu * math.sqrt(d_t) + z_alpha/2 * ModCov * B * std_annu**2 * d_t))
print("daily Duration-Convexity-based VaR:", round(VaR_DC, 4))

monthly Duration-based VaR: 11.2219
monthly Duration-Convexity-based VaR: 10.1423

daily Duration-based VaR: 2.5126
daily Duration-Convexity-based VaR: 2.4612


In [4]:
# Normal Bond Price Change

P = 100
T = 25
y = 0.09
c_rate = 0.06
B = BondPrice(P, T, c_rate, y)

d_y = 100 / 10000
d_t = 0


[MacDur, ModDur, ModCov] = myBondCalc(B, P, T, c_rate, y)

# duration-based approximate price change
D_price_change = B * y * d_t - ModDur * B * d_y
print("Duration-based price change:", round(D_price_change, 4))

# duration-convexity-based approximate price change
DC_price_change = B * y * d_t - ModDur * B * d_y + ModCov * B * (d_y ** 2) / 2
print("Duration-Convexity-based price change:", round(DC_price_change, 4))

Duration-based price change: -7.435
Duration-Convexity-based price change: -6.7987


In [5]:
# portfolio Dur and Cov, hedge
# --- * form 1 * --- #
# P1 = 100
# T1 = 20
# y1 = 0.03
# c_rate1 = 0.0
# B1 = BondPrice(P1, T1, c_rate1, y1)

# P2 = 100
# T2 = 30
# y2 = 0.04
# c_rate2 = 0.0
# B2 = BondPrice(P2, T2, c_rate2, y2)


# [MacDur1, ModDur1, ModCov1] = myBondCalc(B1, P1, T1, c_rate1, y1)
# [MacDur2, ModDur2, ModCov2] = myBondCalc(B2, P2, T2, c_rate2, y2)

# --- * form 2 * --- #
B1 = 100
B2 = -80

ModDur1 = 5
ModDur2 = 1

ModCov1 = 180
ModCov2 = 100


# --- *  * --- #
# portfolio duration
port = B1 + B2
ModDur_port = B1/port * ModDur1 + B2/port * ModDur2
DDur_port = ModDur_port * port
print("Portfolio Duration:", round(ModDur_port, 4))
print("Portfolio Dollar Duration:", round(DDur_port, 4))

# portfolio convexity
ModCov_port = B1/port * ModCov1 + B2/port * ModCov2
DCov_port = ModCov_port * port
print("Portfolio Convexity:", round(ModCov_port, 4))
print("Portfolio Dollar Duration:", round(DCov_port, 4))
print()


# hedge info
P = 100
T = 30
y = 0.05
c_rate = 0
B = BondPrice(P, T, c_rate, y)
[_, ModDur_ZCB, ModCov_ZCB] = myBondCalc(B, P, T, c_rate, y)

# hedge portfolio with ZCB - Dur based
DurH_position = -port * ModDur_port / ModDur_ZCB
print("Duration-based hedge position:", round(DurH_position, 4))

Cov_port_afterDurH = (DCov_port + DurH_position * ModCov_ZCB) / port
print("Convexity after Duration Hedge:", round(Cov_port_afterDurH, 4))

Portfolio Duration: 21.0
Portfolio Dollar Duration: 420.0
Portfolio Convexity: 500.0
Portfolio Dollar Duration: 10000.0

Duration-based hedge position: -14.7
Convexity after Duration Hedge: -120.0


In [6]:
# portfolio hedge, with Dur and Cov
DDur_port = 20 * 21
DCov_port = 20 * 500

P = 100
T1 = 18; T2 = 33
y1 = 0.03; y2 = 0.04
c_rate = 0



B1 = BondPrice(P, T1, c_rate, y1)
B2 = BondPrice(P, T2, c_rate, y2)
[_, ModDur1, ModCov1] = myBondCalc(B1, P, T1, c_rate, y1)
[_, ModDur2, ModCov2] = myBondCalc(B2, P, T2, c_rate, y2)


# solve with Cholesky Decom
A = np.array([[ModDur1, ModDur2], 
              [ModCov1, ModCov2]])
B = np.array([DDur_port, DCov_port])
x = np.linalg.solve(A, B)
print("position of Bond 1:", round(x[0], 4))
print("position of Bond 2:", round(x[1], 4))

position of Bond 1: 14.9858
position of Bond 2: 4.9829


In [2]:
# option approach on bond valuation and VaR
# V_f can be the trading price on market
V_f = 100
B_fv = 50
T = 5
rf = 0.06
std_f = 0.2
q = 0


V_e = myPriceEuroOption(V_f, B_fv, std_f, q, rf, T, "call")[0]
[V_def, delta_def, gamma_def] = myPriceEuroOption(V_f, B_fv, std_f, q, rf, T, "put")

# V_d = V_f - V_e
V_d = B_fv * math.exp(-rf*T) - V_def
spread = -1/T * math.log(V_d/B_fv) - rf


# valuation and spread
print("Value of equity:", round(V_e, 4))
print("Value of defaultable debt:", round(V_d, 4))
print("spread on the debt:", round(spread * 10000, 2), "bps", "(%s)"%round(spread, 6))
print()


# Delta VaR of debt with option appraoch
z_alpha = norm.ppf(0.05)
VaR_debt = -(0 + z_alpha * abs(delta_def) * (std_f/math.sqrt(252)) * V_f)
print("Daily Delta VaR at 0.05:", round(VaR_debt, 4))
VaR_debt = -(0 + z_alpha * abs(delta_def) * (std_f/math.sqrt(12)) * V_f)
print("Monthly Delta VaR at 0.05:", round(VaR_debt, 4))
print()


# # Delta-Gamma VaR of debt with option appraoch
# z_alpha = norm.ppf(0.05)
# DGVaR_debt = -(0 + z_alpha * abs(delta_def) * (std_f/math.sqrt(252)) * V_f\
#              + (z_alpha**2)/2 * gamma_def * (V_f*std_f)**2 / 252)
# print("Daily Delta-Gamma VaR at 0.05:", round(DGVaR_debt, 4))
# DGVaR_debt = -(0 + z_alpha * abs(delta_def) * (std_f/math.sqrt(12)) * V_f\
#              + (z_alpha**2)/2 * gamma_def * (V_f*std_f)**2 / 12)
# print("Monthly Delta-Gamma VaR at 0.05:", round(DGVaR_debt, 4))

NameError: name 'myPriceEuroOption' is not defined

In [16]:
# exp 3 KMV's Distance to Default
V_e = 3
std_e = 0.8
rf = 0.05
B_fv = 10
T = 1


def d12_calc(V, F, rf, std, T):
    _d1n = math.log(V/F) + (rf + 1/2 * std**2) * T
    _d1d = std * math.sqrt(T)
    _d1 = _d1n / _d1d
    _d2 = _d1 - std * math.sqrt(T)
    return [_d1, _d2]

def KMV(E, std_e, rf, F, T):
    def KMVSolver(x):
        V, std_v = x.tolist()
        [_d1, _d2] = d12_calc(V, F, rf, std_v, T)
        

        lhs1 = E
        rhs1 = V * norm.cdf(_d1) - math.exp(-rf*T) * F * norm.cdf(_d2)

        lhs2 = std_e
        rhs2 = V/E * norm.cdf(_d1) * std_v
    
        return [lhs1 - rhs1,
                lhs2 - rhs2]
    
    [V_f, std_f] = optimize.fsolve(KMVSolver, [B_fv, std_e])
    return [V_f, std_f]


# Debt Valuation
[V_f, std_f] = KMV(E=V_e, std_e=std_e, rf=rf, F=B_fv, T=T)
V_d = V_f - V_e
print("Value of Firm:", round(V_f, 4))
print("Value of Debt:", round(V_d, 4))
print()


# Debt Risk Factors
[d1, d2] = d12_calc(V_f, B_fv, rf, std_f, T)
POD = norm.cdf(-d2)
EL = 1 - V_d / (B_fv * math.exp(-rf * T))
RR = 1 - EL / POD

print("probability of default (POD):", round(POD, 6))
print("expected loss (EL):", round(EL, 6))
print("recovery rate (RR):", round(RR, 6))

Value of Firm: 12.3954
Value of Debt: 9.3954

probability of default (POD): 0.126971
expected loss (EL): 0.01229
recovery rate (RR): 0.903206


In [9]:
TPMat = np.array(
        [[90.81,8.33,0.68,0.06,0.12,0,0,0],
         [0.7,90.65,7.79,0.64,0.06,0.14,0.02,0],
         [0.09,2.27,91.05,5.52,0.74,0.26,0.01,0.06],
         [0.02,0.33,5.95,86.93,5.3,1.17,0.12,0.18],
         [0.03,0.14,0.67,7.73,80.53,8.84,1,1.06],
         [0,0.11,0.24,0.43,6.48,83.47,4.07,5.2],
         [0.22,0,0.22,1.3,2.38,11.24,64.86,19.78],
         [0,0,0,0,0,0,0,100]
        ])
TPMatrix = pd.DataFrame(TPMat)

# [columns]: to; [index]: from
# TPMatrix["to"]["from"]
TPMatrix.columns = ["AAA", "AA", "A", "BBB", "BB", "B", "CCC", "Default"]
TPMatrix.index = ["AAA", "AA", "A", "BBB", "BB", "B", "CCC", "Default"]

TPMatrix

,AAA,AA,A,BBB,BB,B,CCC,Default
AAA,90.81,8.33,0.68,0.06,0.12,0.00,0.00,0.00
AA,0.70,90.65,7.79,0.64,0.06,0.14,0.02,0.00
A,0.09,2.27,91.05,5.52,0.74,0.26,0.01,0.06
BBB,0.02,0.33,5.95,86.93,5.30,1.17,0.12,0.18
BB,0.03,0.14,0.67,7.73,80.53,8.84,1.00,1.06
B,0.00,0.11,0.24,0.43,6.48,83.47,4.07,5.20
CCC,0.22,0.00,0.22,1.30,2.38,11.24,64.86,19.78
Default,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00


In [10]:
Y3_TPMatrix = np.dot(np.dot(TPMatrix, TPMatrix) / 100, TPMatrix / 100)

Y3_TPMatrix = pd.DataFrame(Y3_TPMatrix)
Y3_TPMatrix.columns = ["AAA", "AA", "A", "BBB", "BB", "B", "CCC", "Default"]
Y3_TPMatrix.index = ["AAA", "AA", "A", "BBB", "BB", "B", "CCC", "Default"]
Y3_TPMatrix

,AAA,AA,A,BBB,BB,B,CCC,Default
AAA,75.047353,20.634760,3.474513,0.447483,0.310940,0.068849,0.008608,0.007495
AA,1.750148,75.140710,19.451572,2.696383,0.423209,0.426092,0.058455,0.053432
A,0.270962,5.707504,76.872731,13.360224,2.456628,0.973647,0.087157,0.271147
BBB,0.074220,1.175845,14.365608,67.643501,11.538571,3.828674,0.468013,0.905568
BB,0.081421,0.458672,2.776154,16.600807,54.738129,18.456384,2.455308,4.433125
B,0.029687,0.297661,0.808905,2.373996,13.455880,60.673314,6.944988,15.415569
CCC,0.408120,0.110303,0.723552,2.909198,5.659275,19.291638,28.331687,42.566226
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000


In [11]:
Y3_TPMatrix["AAA"]["BBB"] + Y3_TPMatrix["AA"]["BBB"] + Y3_TPMatrix["A"]["BBB"] + Y3_TPMatrix["BBB"]["BBB"]

83.2591734829